In [1]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

happy = ""
top = -1
    
# Define actions and corresponding labels
actions = ['yes_line', 'no_line']
seq_length = 30

model = load_model('models/model.h5')

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

cap = cv2.VideoCapture(0)

seq = []
action_seq = []

while cap.isOpened():
    ret, img = cap.read()
    img0 = img.copy()
    
    
    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    # cv2.putText(img,f'hi', org=[50,50], fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
    

    if result.multi_hand_landmarks is not None:
        
    
        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 4))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility]
                
                check = joint[5] - joint[17]
                
                
            v1 = joint[[0, 1, 2, 3, 0, 5, 6, 7, 0, 9, 10, 11, 0, 13, 14, 15, 0, 17, 18, 19], :3]
            v2 = joint[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], :3]
            v = v2 - v1
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            angle = np.arccos(np.einsum('nt,nt->n', v[[0, 1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 14, 16, 17, 18], :],
                                        v[[1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14, 15, 17, 18, 19], :]))
            angle = np.degrees(angle)

            d = np.concatenate([joint.flatten(), check])
            d = np.concatenate([d, angle])

                    #d = np.concatenate([joint.flatten(), angle_label])
                    #d = np.concatenate([d, check])
                    
            
            print(d.shape)
            
            seq.append(d)

            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            if len(seq) < seq_length:
                continue
                
            print(d[84])

            input_data = np.expand_dims(np.array(seq[-seq_length:], dtype=np.float32), axis=0)
            print(input_data.shape)
            y_pred = model.predict(input_data).squeeze()
            print(y_pred)

            i_pred = int(np.argmax(y_pred))
            conf = y_pred[i_pred]
            
#             cv2.putText(img, 'yes!!', org=[25,50],
#                     fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

            if conf < 0.95:
                continue

            action = actions[i_pred]
            action_seq.append(action)

            if len(action_seq) < 3:
                continue

            this_action = '?'
            if action_seq[-1] == action_seq[-2] == action_seq[-3]: # 정확한 액션이라면
                this_action = action
                
                #org=(int(res.landmark[0].x * img.shape[1])
                
            
            print(this_action)
            
            if cv2.waitKey(1) == ord('a'):
                if this_action == 'space':
                    this_action = " "
                    happy = happy + this_action
                    top +=1
                    
                if this_action =='backspace':
                    happy = happy[:-1]
                    print(happy)
                    
                else:
                    happy = happy + this_action.upper() # upper() 대문자 변환 함수
                    top += 1
                
            cv2.putText(img, f'{this_action.upper()}', org=[25,50],
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
          
            # happy = ''.join(str(_) for _ in happy)
            print(type(happy))
             
            cv2.putText(img , happy , org=[25,450], fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)        
            cv2.imshow('img', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


2023-07-17 14:07:50.761395: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[ WARN:0@12.905] global /Users/ec2-user/ci_py311/opencv-suite_1678377996177/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
(103,)
-0.09994763135910034
(1, 30, 103)
1/1 [==============================] - 0s 325ms/step
[1.3460907e-12 1.0000000e+00]
(103,)
-0.09478580951690674
(1, 30, 103)
1/1 [==============================] - 0s 42ms/step
[1.2309124e-12 1.0000000e+00]
(103,)
-0.0952901542186737
(1, 30, 103)
1/1 [==============================] - 0s 51ms/step
[1.8520066e-12 1.0000000e+00]
no_line
<class 'str'>
(103,)
-0.11119091510772705
(1, 30, 103)
1/1 [==============================] - 0s 31ms/step
[3.7466046e-12 1.0000000e+00]
no_line
<class 'str'>
(103,)
-0.10871881246566772
(1, 30, 103)
1/1 [==============================] - 0s 34ms/step
[4.2331062e-11 1.0000000e+00]
no_line
<class 'str'>
(103,)
-0.10728877782821655
(1, 30, 103)
1/1 [==============================] - 0s 29ms/step
[4.0337708e-10 1.000000

1/1 [==============================] - 0s 33ms/step
[9.999100e-01 9.004988e-05]
yes_line
<class 'str'>
(103,)
-0.09906870126724243
(1, 30, 103)
1/1 [==============================] - 0s 32ms/step
[9.999106e-01 8.943859e-05]
yes_line
<class 'str'>
(103,)
-0.07715970277786255
(1, 30, 103)
1/1 [==============================] - 0s 32ms/step
[9.999393e-01 6.063492e-05]
yes_line
<class 'str'>
(103,)
0.0901268720626831
(1, 30, 103)
1/1 [==============================] - 0s 29ms/step
[9.999368e-01 6.321304e-05]
yes_line
<class 'str'>
(103,)
0.09609079360961914
(1, 30, 103)
1/1 [==============================] - 0s 29ms/step
[9.999325e-01 6.752216e-05]
yes_line
<class 'str'>
(103,)
0.09414520859718323
(1, 30, 103)
1/1 [==============================] - 0s 28ms/step
[9.9990976e-01 9.0277725e-05]
yes_line
<class 'str'>
(103,)
0.09117746353149414
(1, 30, 103)
1/1 [==============================] - 0s 37ms/step
[9.9976701e-01 2.3300896e-04]
yes_line
<class 'str'>
(103,)
0.09147414565086365
(1, 30,

1/1 [==============================] - 0s 27ms/step
[4.6276935e-05 9.9995375e-01]
no_line
<class 'str'>
(103,)
-0.09632211923599243
(1, 30, 103)
1/1 [==============================] - 0s 27ms/step
[1.8842466e-04 9.9981159e-01]
no_line
<class 'str'>
(103,)
-0.09531813859939575
(1, 30, 103)
1/1 [==============================] - 0s 28ms/step
[6.637754e-04 9.993362e-01]
no_line
<class 'str'>
(103,)
-0.10713672637939453
(1, 30, 103)
1/1 [==============================] - 0s 33ms/step
[0.00261329 0.99738675]
no_line
<class 'str'>
(103,)
-0.097115159034729
(1, 30, 103)
1/1 [==============================] - 0s 32ms/step
[0.0049515  0.99504846]
no_line
<class 'str'>
(103,)
-0.09309977293014526
(1, 30, 103)
1/1 [==============================] - 0s 32ms/step
[0.00575436 0.9942457 ]
no_line
<class 'str'>
(103,)
-0.092987060546875
(1, 30, 103)
1/1 [==============================] - 0s 31ms/step
[0.00684559 0.9931544 ]
no_line
<class 'str'>
(103,)
-0.09255242347717285
(1, 30, 103)
1/1 [==========